## Microsoft Building Footprints

Bing Maps is releasing open building footprints around the world. The building footprints are detected using Bing Maps imagery between 2014 and 2021 including Maxar and Airbus imagery. The data is freely available for download and use under ODbL.

Our aim is to use this dataset to pretrain a robust building detection model from satelite image, which can then be fine-tuned for specific use-case

In [ ]:
import os
from IPython.display import clear_output

# https://github.com/opengeos/leafmap/blob/e35e0a75a125614244e5913755c50ec4f307bcab/docs/notebooks/74_map_tiles_to_geotiff.ipynb#L7
# require reload after installation
%pip install -U leafmap
clear_output()

# restart kernel
import os
os._exit(0)

In [1]:
import planetary_computer
import pystac_client
import dask.dataframe
import dask_geopandas
import dask.distributed
import deltalake
import shapely.geometry
import contextily
import mercantile

### Data access

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) to get a link to the assets. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

### Using Delta Table Files

The assets are a set of [geoparquet](https://geoparquet.org/) files grouped by a processing date. Newer files (since April 25th, 2023) are stored in [Delta Format](https://docs.delta.io/latest/delta-intro.html). This is a layer on top of parquet files offering scalable metadata handling, which is useful for this dataset.

In [3]:
asset = collection.assets["delta"]

This Delta Table is partitioned by `RegionName` and `quadkey`. Each `(RegionName, quadkey)` pair will contain one or more parquet files (depending on how dense that particular quadkey) is. The quadkeys are at level 9 of the [Bing Maps Tile System](https://learn.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system).

In [4]:
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

### (Optional) Examples of using the Delta Table

You can load all the files for a given `RegionName` with a query like this:

In [ ]:
# file_uris = table.file_uris([("RegionName", "=", "VaticanCity")])

# df = dask_geopandas.read_parquet(file_uris, storage_options=storage_options)
# df

In [ ]:
# ax = df.compute().plot(figsize=(12, 12), color="yellow")
# contextily.add_basemap(
#     ax,
#     source=contextily.providers.Esri.WorldGrayCanvas,
#     crs=df.crs.to_string(),
#     zoom=16,
# )
# ax.set_axis_off()

This partitioning, and the Delta Table format, let you quickly query the subset of files that match your area of interest.

For example, we can visualize the footprints for a small region outside the town of Ueckermünde in Northeast Germany defined by this bounding box.

In [ ]:
# area_of_interest = shapely.geometry.box(14.11, 53.73, 14.13, 53.75)

You will need to translate your area of interest to a set of intersecting quadkeys, which can be done with [mercantile](https://mercantile.readthedocs.io/en/stable/).

In [ ]:
# quadkeys = [
#     int(mercantile.quadkey(tile))
#     for tile in mercantile.tiles(*area_of_interest.bounds, zooms=9)
# ]
# quadkeys

Now we can provide those quadkeys as a partition filter to get the set of matching URIs.

In [ ]:
# uris = table.file_uris([("RegionName", "=", "Germany"), ("quadkey", "in", quadkeys)])
# uris

And quickly load them into, for example, a dask-geopandas `GeoDataFrame`:

In [ ]:
# df = dask_geopandas.read_parquet(uris, storage_options=storage_options)
# df

### Utilites function to Extract Bounding Boxes

In [5]:
import pandas as pd
import leafmap
import os
import numpy as np
from PIL import Image # ImageDraw
# from itertools import chain
from IPython.display import clear_output
# cv2 cannot be used here

In [6]:
#------------------------------------------------
# get bounding box of buildings in the city
#------------------------------------------------
def getBuildingBbox(city):
    file_uris = table.file_uris([("RegionName", "=", city)])
    # file_uris = table.file_uris([("quadkey", "=", city)])
    df = dask_geopandas.read_parquet(file_uris, storage_options=storage_options)
    
    return df

#------------------------------------------------
# get the bounding box of the entire city
#------------------------------------------------
def getRegionBbox(df):
    # get region bbox
    bounded_df = df.compute().bounds 
    _, _, maxx, maxy = bounded_df.max()
    minx, miny, _, _ = bounded_df.min()
    region_bbox = [minx, miny, maxx, maxy]

    return region_bbox

In [7]:
#------------------------------------------------
# crop the region into tiles
#------------------------------------------------
def cropRegion(region_bbox, step):
    # region bounding box
    minx, miny, maxx, maxy = region_bbox
    
    # store all bbox in each tile
    region_bbox_list = []
    
    # handle large image situation, crop into tiles
    if (maxx - minx) > step or (maxy - miny) > step:
        new_minx, new_miny = minx, miny

        num_x_tiles = int((maxx - minx)//step + 1)
        num_y_tiles = int((maxy - miny)//step + 1)
        
        print(f'Number of x tiles: {num_x_tiles}')
        print(f'Number of y tiles: {num_y_tiles}')
        
        print(f'\nTotal number of tiles: {num_x_tiles*num_y_tiles}')
        for i in range(num_y_tiles):
            for j in range(num_x_tiles):
                new_maxx, new_maxy = new_minx + step, new_miny + step
                
                region_bbox = [new_minx, new_miny, new_maxx, new_maxy]
                region_bbox_list.append(region_bbox)
                
                new_minx = new_maxx

            new_minx = minx
            new_miny = new_maxy
    
    # if small image, then treat the entire image as one tile
    else:
        region_bbox_list.append(region_bbox)
    
    return region_bbox_list

In [8]:
#------------------------------------------------
# An assumption that each building exist only within one of the tiles
# to save computational power & time
#------------------------------------------------
def createList(row, num_x_tiles, step):
    return int(row['left_tile']+row['top_tile']*num_x_tiles)

In [9]:
#------------------------------------------------

#------------------------------------------------
def labelTileRegion(region_bbox, bounded_df, step):
    minx, miny, maxx, maxy = region_bbox
    
    bounded_df['left_tile'] = (bounded_df['minx'] - minx)//step
    bounded_df['right_tile'] = (bounded_df['maxx'] - minx)//step + 1
    bounded_df['top_tile'] = (bounded_df['miny'] - miny)//step
    bounded_df['bottom_tile'] = (bounded_df['maxy'] - miny)//step + 1
    
    # bounded_df['left_tile'] = bounded_df['left_tile'].astype(int)
    # bounded_df['right_tile'] = bounded_df['right_tile'].astype(int)
    # bounded_df['top_tile'] = bounded_df['top_tile'].astype(int)
    # bounded_df['bottom_tile'] = bounded_df['bottom_tile'].astype(int)
    
    num_x_tiles = int((maxx - minx)//step + 1)
    num_y_tiles = int((maxy - miny)//step + 1)
    
    bounded_df['Tile'] = bounded_df.apply(lambda row: createList(row, num_x_tiles, step), axis=1)
    
    return bounded_df

In [10]:
#------------------------------------------------
# Filtering: we only want buildings which are inside the region (city)
#------------------------------------------------
def filterDf(region_bbox, bounded_df):
    minx, miny, maxx, maxy = region_bbox
    
    new_bounded_df = bounded_df.copy()
    
    new_bounded_df['minx'] = new_bounded_df['minx'].apply(lambda x: minx if x < minx else x)
    new_bounded_df['minx'] = new_bounded_df['minx'].apply(lambda x: np.nan if x > maxx else x)
    new_bounded_df['maxx'] = new_bounded_df['maxx'].apply(lambda x: maxx if x > maxx else x)
    new_bounded_df['maxx'] = new_bounded_df['maxx'].apply(lambda x: np.nan if x < minx else x)
    
    new_bounded_df['miny'] = new_bounded_df['miny'].apply(lambda x: miny if x < miny else x)
    new_bounded_df['miny'] = new_bounded_df['miny'].apply(lambda x: np.nan if x > maxy else x)
    new_bounded_df['maxy'] = new_bounded_df['maxy'].apply(lambda x: maxy if x > maxy else x)
    new_bounded_df['maxy'] = new_bounded_df['maxy'].apply(lambda x: np.nan if x < miny else x)
    
    new_bounded_df = new_bounded_df.dropna(how='any')
    
    return new_bounded_df

In [11]:
#------------------------------------------------
# normalize the bounding box into YOLO annotation format:
# class, x center, y center, width, height
#------------------------------------------------
def normaliseBbox(region_bbox, region_bbox_list, df, step):
    tile_bbox_list = []
    filtered_region_bbox_list = []
    bounded_df = df.bounds
    bounded_df = labelTileRegion(region_bbox, bounded_df, step)
    
    # region_list = set(chain.from_iterable(row['Tile'] for _, row in bounded_df.iterrows()))
    region_list = set(bounded_df['Tile'].compute().to_list())
    
    # Testing purpose
    # region_list = [1242]
    # print(bounded_df.head(5))
    # print(region_bbox_list[region_list[0]])
    
    print(f'Number of region tiles after filtering: {len(region_list)}')
    print('\nGenerating tiles...')
    region_list = list(region_list)[290:300] # change here
    for count, region in enumerate(region_list):
        count += 290 # change here
        print(f'\n~ Tile {count}')
        region_bbox = region_bbox_list[region]
        new_bounded_df = filterDf(region_bbox, bounded_df)
        cropped_minx, cropped_miny, cropped_maxx, cropped_maxy = region_bbox

        # normalise bounding box
        # get building world map bbox
        building_minx = new_bounded_df['minx']
        building_miny = new_bounded_df['miny']
        building_maxx = new_bounded_df['maxx']
        building_maxy = new_bounded_df['maxy']

        # normalized building bbox to the region (0 to 1 range)
        new_bounded_df['building_minx'] = (building_minx - cropped_minx) / (cropped_maxx - cropped_minx)
        new_bounded_df['building_miny'] = (building_miny - cropped_miny) / (cropped_maxy - cropped_miny)
        new_bounded_df['building_maxx'] = (building_maxx - cropped_minx) / (cropped_maxx - cropped_minx)
        new_bounded_df['building_maxy'] = (building_maxy - cropped_miny) / (cropped_maxy - cropped_miny) 

        # flip y axis, due to some reason
        new_bounded_df['building_miny'] = 1 - new_bounded_df['building_miny']
        new_bounded_df['building_maxy'] = 1 - new_bounded_df['building_maxy']
        new_bounded_df['building_miny'], new_bounded_df['building_maxy'] = new_bounded_df['building_maxy'], new_bounded_df['building_miny']

        # get bbox
        new_bounded_df['class'] = 0
        new_bounded_df['x'] = (new_bounded_df['building_maxx'] + new_bounded_df['building_minx']) / 2
        new_bounded_df['y'] = (new_bounded_df['building_maxy'] + new_bounded_df['building_miny']) / 2
        new_bounded_df['w'] = new_bounded_df['building_maxx'] - new_bounded_df['building_minx']
        new_bounded_df['h'] = new_bounded_df['building_maxy'] - new_bounded_df['building_miny']

        new_bounded_df = new_bounded_df[(new_bounded_df['w'] != 0.0) & (new_bounded_df['h'] != 0.0)]

        if len(new_bounded_df.index) != 0:
            new_df = new_bounded_df[['class', 'x', 'y', 'w', 'h']]
            tile_bbox_list.append(new_df)
            filtered_region_bbox_list.append(region_bbox)
            
            # saving while filtering
            saveBbox([new_df], city, count)
            saveRegionTif(city, [region_bbox], count, zoom=20)
            
            clear_output()
            
    print('Done filtering and normalising.')
    
    return tile_bbox_list, filtered_region_bbox_list

In [12]:
#------------------------------------------------
# save bounding box into anntotaion txt files
#------------------------------------------------
def saveBbox(tile_bbox_list, city, count):
    output_directory = './region_labels'
    output_city_directory = f'./region_labels/{city}'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    if not os.path.exists(output_city_directory):
        os.makedirs(output_city_directory)
    
    if not isinstance(tile_bbox_list, list):
        tile_bbox_list = [tile_bbox_list]
    
    print('\nGenerating labels...')
    for building_df in tile_bbox_list:
        file_name = f'{city}_{str(count).zfill(5)}.txt'

        with open(f'./region_labels/{city}/{file_name}', 'w') as f:
            df_string = building_df.compute().to_string(header=False, index=False)
            f.write(df_string)

    print(f"{city}'s label saved.\n")

In [13]:
#------------------------------------------------
# save region tif
#------------------------------------------------
def saveRegionTif(city, region_bbox_list, count, zoom=20):
    output_directory = './region_tif'
    output_city_directory = f'./region_tif/{city}'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    if not os.path.exists(output_city_directory):
        os.makedirs(output_city_directory)
    
    # get tif image
    print(f'A total of {len(region_bbox_list)} images will be saved.')
    for region_bbox in region_bbox_list:
        minx, miny, maxx, maxy = region_bbox
    
        output_path = output_city_directory + f'/{city}_{str(count).zfill(5)}.tif'
        leafmap.map_tiles_to_geotiff(output_path, region_bbox, zoom=zoom, source='SATELLITE')
    
    print(f"{city}'s tif file saved.")

In [14]:
#------------------------------------------------
# convert tiff to jpg
#------------------------------------------------
def convert_tiff_to_jpeg(input_dir, output_dir):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))

            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'

            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG')
            
    print("Conversion from TIFF to JPEG completed.")

### Start extraction

In [15]:
#------------------------------------------------
# main function
#------------------------------------------------
for city in ['PuertoRico']:
    # which city
    print(f'=== {city} ===')
    
    # get all bounding box of buildings in this city
    df = getBuildingBbox(city)
    
    # get the bounding box of the entire city itself
    region_bbox = getRegionBbox(df)
    print(f'Region Bbox: {region_bbox}')
    
    step = 0.01
    region_bbox_list = cropRegion(region_bbox, step)

    tile_bbox_list, filtered_region_bbox_list = normaliseBbox(region_bbox, region_bbox_list, df, step)
    
#     if len(tile_bbox_list) != 0:
#         saveBbox(tile_bbox_list, city)

#         saveRegionTif(city, filtered_region_bbox_list, zoom=20)
#         convert_tiff_to_jpeg(f'./region_tif/{city}/', f'./region_jpeg/{city}/')

    print('Converting TIFF to JPEG...')
    convert_tiff_to_jpeg(f'./region_tif/{city}/', f'./region_jpeg/{city}/')

print('Done!')

Done filtering and normalising.
Converting TIFF to JPEG...
Conversion from TIFF to JPEG completed.
Done!


In [ ]:
# zip file for downloading
# !zip -r ./region_tif.zip ./region_tif
!zip -r ./region_jpeg.zip ./region_jpeg
!zip -r ./region_labels.zip ./region_labels/

**Afternote**

Upload all the images and labels to Roboflow for converting to XML format.</br>
Run colab GenerateTile.ipynb file to generate tiles for each image.</br>
Upload back to Roboflow for convertion to YOLO format and model training.</br>